In [ ]:
%%capture
!sudo apt update && sudo apt install espeak ffmpeg libespeak1
!pip install gTTS

In [4]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
%%capture
!cp -r /content/drive/MyDrive/521\ ML\ \ final\ project/dataset/iam/compressed /
!rm -r /words
!mkdir -p /words
!tar xvzf /compressed/words.tgz -C /words
!rm -r /lines
!mkdir -p /lines
!tar xvzf /compressed/lines.tgz -C /lines
!rm -r /forms
!mkdir -p /forms
!tar xvzf /compressed/formsA-D.tgz -C /forms
!tar xvzf /compressed/formsE-H.tgz -C /forms
!tar xvzf /compressed/formsI-Z.tgz -C /forms
!rm -r /words_label
!mkdir -p /words_label
!tar xvzf /compressed/ascii.tgz -C /words_label
!rm -r /xml
!mkdir -p /xml
!tar xvzf /compressed/xml.tgz -C /xml
!rm -r /samples
!mkdir -p /samples
!cp -r /content/drive/MyDrive/521\ ML\ \ final\ project/dataset/iam/samples/* /samples/


In [76]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import Counter
from skimage.filters import sobel 
from numpy.lib.type_check import imag
import random
from sklearn.model_selection import train_test_split,cross_val_score,KFold,cross_validate
from sklearn.metrics import recall_score,f1_score,precision_score,accuracy_score,confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import string
from gtts import gTTS
from IPython.display import Audio


In [77]:
np.random.seed(2022)


In [78]:
def get_word(line):
  return " ".join(line.split("\n")[0].split(" ")[8:]).strip()

In [79]:
def get_unique_words_and_counter(file_path):
  unique_words = defaultdict(list)
  counter_input_data = []
  word_file = open(file_path, "r").readlines()
  for line in word_file:
      if line[0] == "#":
          continue
      if line.split(" ")[1] != "err":
          word = get_word(line)
          unique_words[word].append(line.strip())
          counter_input_data.append(word)
  return unique_words,counter_input_data


In [80]:
def filter_labels(most_common_words,unique_words,max_words,max_images_per_word): 
  filtered_labels = []
  num_word = 0
  for word,size in most_common_words:
    flag = False
    for char in word:
      if char in string.ascii_letters:
        flag=True
        break
    if flag:
      if num_word < max_words:
        if size >= max_images_per_word:
            filtered_labels.extend(unique_words[word][0:max_images_per_word])
            num_word+=1
      else:
        break        
  return filtered_labels


In [81]:
def get_labels_for_the_words(unique_words,words,max_images_per_word):
  labels = []
  for word in unique_words:
    if word in words:
      labels.extend(unique_words[word][0:max_images_per_word])
  return labels

In [82]:
def get_input_for_labels_based_on_training_words(file_path,train_words,max_images_per_word):
  unique_words,counter_input_data = get_unique_words_and_counter(file_path)
  input_for_labels = get_labels_for_the_words(unique_words,train_words,max_images_per_word)
  return input_for_labels

In [83]:
def get_input_for_labels_based_max_size(file_path,max_words=50,max_images_per_word=100):
  unique_words,counter_input_data = get_unique_words_and_counter(file_path)
  counter_data=Counter(counter_input_data)    
  most_common_words = counter_data.most_common()
  input_for_labels = filter_labels(most_common_words,unique_words,max_words,max_images_per_word)
  return input_for_labels

In [84]:
def read_image(image_path,color=False):
    if color:
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      h, w, c = img.shape

      if w > 1000:
          
          new_w = 1000
          ar = w/h
          new_h = int(new_w/ar)
          
          img = cv2.resize(img, (new_w, new_h), interpolation = cv2.INTER_AREA)
    else:
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    return img


In [85]:
def process_image(img):
    resized_img = cv2.resize(img,IMAGE_SIZE)
    image_array = np.array(resized_img).flatten()
    return image_array

In [86]:
def show_image(image):
  plt.imshow(image, cmap='gray')

In [87]:
def get_images_and_labels(base_path, samples,IMAGE_SIZE):
    base_image_path = os.path.join(base_path, "/words")
    data = []
    for (i, file_line) in enumerate(samples):
        line_split = file_line.strip()
        line_split = line_split.split(" ")
        image_name = line_split[0]
        partI = image_name.split("-")[0]
        partII = image_name.split("-")[1]
        img_path = os.path.join(
            base_image_path, partI, partI + "-" + partII, image_name + ".png"
        )
        if os.path.getsize(img_path):
            img = read_image(img_path)
            image_array = process_image(img)
            label = get_word(file_line)
            data.append((image_array,label))
    return data
  

In [88]:

def get_features_and_labels(input_data, label_encoder):
  tmp_features = []
  tmp_labels =[]
  for feature,label in input_data:
      tmp_features.append(feature)
      tmp_labels.append(label)
  features = np.array(tmp_features)
  labels =label_encoder.fit_transform(np.array(tmp_labels))
  return features,labels

In [89]:
def get_gaussian_model(model_name):
  models = {
      'GaussianNB':GaussianNB,
      'MultinomialNB':MultinomialNB

  }
  model = models[model_name]()  
  return model

def get_random_forest_model(n_estimators=100):
  model = RandomForestClassifier(n_estimators=n_estimators)
  return model

def get_knn_model(n_neighbors=5,metric='minkowski',p=2): 
    model = KNeighborsClassifier(n_neighbors = n_neighbors, metric =metric, p = p)
    return model

def get_svm_model(poly_degree=3,poly_c=1):
  model = svm.SVC(kernel='poly', degree=poly_degree, C=poly_c)
  return model

def get_predict_from_model(model,x_test):
    y_pred = model.predict(x_test)
    return y_pred

def get_accuracy_score(model,x_test,y_test,model_name):
    y_pred = get_predict_from_model(model,x_test)
    acc_score = accuracy_score(y_test, y_pred)
    # poly_f1 = f1_score(y_test, poly_pred, average='weighted')
    # print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
    # cm = confusion_matrix(y_test, y_pred)
    print('Model: {model_name}  Accuracy score: {acc_score}'.format(model_name=model_name,acc_score=acc_score))
    return y_pred

def run_model(model,x_train, y_train,x_test,y_test,model_name):
  model.fit(x_train, y_train)
  return get_accuracy_score(model,x_test,y_test,model_name)


def run_model_with_cv(model_name,model, x, y,n_splits=5):
    kf=KFold(n_splits=n_splits)
    scoring = {'accuracy':'accuracy','f1':'f1_weighted','precision':'precision_weighted',
               'recall':'recall_weighted'}
    results = cross_validate(estimator=model,
                              X=x,
                              y=y,
                              cv=kf,
                              scoring=scoring,
                              return_train_score=True)
    data ={
            # "Training Accuracy scores": results['train_accuracy'],
            "Mean Training Accuracy": results['train_accuracy'].mean()*100,
            # "Training Precision scores": results['train_precision'],
            "Mean Training Precision": results['train_precision'].mean(),
            # "Training Recall scores": results['train_recall'],
            "Mean Training Recall": results['train_recall'].mean(),
            # "Training F1 scores": results['train_f1'],
            "Mean Training F1 Score": results['train_f1'].mean(),
            # "Validation Accuracy scores": results['test_accuracy'],
            "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
            # "Validation Precision scores": results['test_precision'],
            "Mean Validation Precision": results['test_precision'].mean(),
            # "Validation Recall scores": results['test_recall'],
            "Mean Validation Recall": results['test_recall'].mean(),
            # "Validation F1 scores": results['test_f1'],
            "Mean Validation F1 Score": results['test_f1'].mean()
            }
    print(model_name,data)
    return data



In [90]:
def get_predictions(label_encoder,results):
  labels_inversed = label_encoder.inverse_transform(results)
  return labels_inversed

def show_predictions(labels,x_test,sample=5):
  total = len(labels[0:sample])
  breaker = 4
  _, ax = plt.subplots((total//4)+1,4,figsize=(15, 8))
  results = []
  for idx,img_ar in enumerate(x_test[0:sample]):
      results.append(labels[idx])
      img = np.round(img_ar * 255).astype(int).reshape(128,128)
      title = f"Prediction: {labels[idx]}"
      ax[idx // breaker, idx % breaker].imshow(img, cmap="gray")
      ax[idx // breaker, idx % breaker].set_title(title)
      ax[idx // breaker, idx % breaker].axis("off")
  for idx in range((total//4+1) *4):
     ax[idx // breaker, idx % breaker].axis("off")
  plt.show()
  return results

In [91]:
def thresholding(image):
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    # plt.imshow(thresh, cmap='gray')
    return thresh

def get_bounding_box_from_image(file_name):
  img = read_image(file_name,color=True)
  # plt.imshow(img);
  thresh_img = thresholding(img);
  # #dilation
  kernel = np.ones((3,85), np.uint8)
  dilated = cv2.dilate(thresh_img, kernel, iterations = 1)
  # plt.imshow(dilated, cmap='gray');
  (contours, heirarchy) = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  sorted_contours_lines = sorted(contours, key = lambda ctr : cv2.boundingRect(ctr)[1]) # (x, y, w, h)
  #dilation
  kernel = np.ones((3,15), np.uint8)
  dilated2 = cv2.dilate(thresh_img, kernel, iterations = 1)
  # plt.imshow(dilated2, cmap='gray');
  img3 = img.copy()
  bounding_boxes = []

  for line in sorted_contours_lines:
      
      # roi of each line
      x, y, w, h = cv2.boundingRect(line)
      roi_line = dilated2[y:y+h, x:x+w]
      
      # draw contours on each word
      (cnt, heirarchy) = cv2.findContours(roi_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      sorted_contour_words = sorted(cnt, key=lambda cntr : cv2.boundingRect(cntr)[0])
      
      for word in sorted_contour_words:
          
          if cv2.contourArea(word) < 400:
              continue
          
          x2, y2, w2, h2 = cv2.boundingRect(word)
          bounding_boxes.append([x+x2, y+y2, x+x2+w2, y+y2+h2])
          cv2.rectangle(img3, (x+x2, y+y2), (x+x2+w2, y+y2+h2), (0, 255, 0),2)
  show_image(img3)
  return bounding_boxes




In [92]:
def get_words_from_image(image_path):
  bounding_boxes = get_bounding_box_from_image(image_path)
  word_images = []
  img = read_image(image_path)
  for bounding_box in bounding_boxes:
    x, y, w, h = bounding_box
    cropped_img = img[y:h,x:w].copy()
    word_image = process_image(cropped_img)
    word_images.append(word_image)
  return word_images 

In [93]:
def convert_to_audio_using_gtts(text):
  tts = gTTS(text,lang='en',slow=True)
  file_name='1.wav' 
  tts.save(file_name)
  return file_name

def convert_to_audio(words,engine_type='gtts'):
  if engine_type=='gtts':
      return convert_to_audio_using_gtts(" ".join(words))
  

In [94]:
def get_word_count(label_encoder,y_train,y_test):
  words = []
  for i in get_predictions(label_encoder,y_train):
    words.append(i)
  for i in get_predictions(label_encoder,y_test):
    words.append(i)
  Counter(words)

In [95]:
def recognize(model,image_path,show_sample=10):
  show_image(cv2.imread(image_path))
  words = np.array(get_words_from_image(image_path))/NORMALIZE_SCALE
  preds = get_predict_from_model(model,words)
  predicted_labels = get_predictions(label_encoder,preds)
  sound_file = convert_to_audio(predicted_labels)
  show_predictions(predictions,words,sample=show_sample)
  return sound_file, predicted_labels

In [96]:
MAX_WORDS = 100
MAX_IMAGES_PER_WORD = 200
TEST_DATASET_SIZE = 0.05
NORMALIZE_SCALE = 255.0
IMAGE_SIZE = (128,128)
TRAIN_WORDS = {'the', 'he', 'for', 'stop', 'on', 'Foot', 'Labour', 'at', 'Griffiths', 'and', 'of', 'by', 'any', '.', 'subject', 'Peers', 'tomorrow', 'nominating', 'has', 'Mr.', 'backed', 'MOVE', 'life', 'be', 'is', 'resolution', 'Ps', 'meeting', 'a', 'from', 'Michael', 'Gaitskell', 'more', 'put', 'Exchange', 'Manchester', 'to', 'down', 'Will', ',', 'made', 'P', 'A'}

In [97]:

base_path_for_images = "/words"
file_path = "/words_label/words.txt"


In [98]:
input_for_labels = get_input_for_labels_based_on_training_words(file_path,TRAIN_WORDS,MAX_IMAGES_PER_WORD)
input_data = get_images_and_labels(base_path_for_images, input_for_labels,
                                   IMAGE_SIZE)

random.shuffle(input_data)

label_encoder = LabelEncoder()
features,labels = get_features_and_labels(input_data,label_encoder)

x_train,x_test,y_train,y_test = train_test_split(features,labels,
                                                 test_size=TEST_DATASET_SIZE)

x_train,x_test = x_train/NORMALIZE_SCALE,x_test/NORMALIZE_SCALE

features = features/NORMALIZE_SCALE
print(len(x_train),len(x_test))

4063 214


In [ ]:
get_word_count(label_encoder,y_train,y_test)

In [99]:
results = None
model_names = [
      'GaussianNB',
      'MultinomialNB'
]
for model_name in model_names:
  nb_model = get_gaussian_model(model_name)
  results = run_model(nb_model,x_train, y_train,x_test,y_test,model_name)
  run_model_with_cv(model_name,nb_model,x_train, y_train)



Model: GaussianNB  Accuracy score: 0.5186915887850467


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

GaussianNB {'Training Accuracy scores': array([0.64830769, 0.64061538, 0.64615385, 0.63795755, 0.65579822]), 'Mean Training Accuracy': 64.57665381066181, 'Training Precision scores': array([0.68666453, 0.68468765, 0.69696946, 0.68043367, 0.69480002]), 'Mean Training Precision': 0.6887110653171352, 'Training Recall scores': array([0.64830769, 0.64061538, 0.64615385, 0.63795755, 0.65579822]), 'Mean Training Recall': 0.6457665381066181, 'Training F1 scores': array([0.64310312, 0.64248345, 0.64353651, 0.63531453, 0.65401131]), 'Mean Training F1 Score': 0.6436897824646938, 'Validation Accuracy scores': array([0.56949569, 0.57195572, 0.53628536, 0.53078818, 0.57389163]), 'Mean Validation Accuracy': 55.64833160646877, 'Validation Precision scores': array([0.60962479, 0.61324873, 0.58678749, 0.5715949 , 0.62076268]), 'Mean Validation Precision': 0.6004037163398517, 'Validation Recall scores': array([0.56949569, 0.57195572, 0.53628536, 0.53078818, 0.57389163]), 'Mean Validation Recall': 0.55648

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is 

MultinomialNB {'Training Accuracy scores': array([0.67046154, 0.66492308, 0.67353846, 0.67886804, 0.66871732]), 'Mean Training Accuracy': 67.13016870548707, 'Training Precision scores': array([0.6974454 , 0.69685096, 0.70115241, 0.704975  , 0.69665565]), 'Mean Training Precision': 0.6994158867707749, 'Training Recall scores': array([0.67046154, 0.66492308, 0.67353846, 0.67886804, 0.66871732]), 'Mean Training Recall': 0.6713016870548707, 'Training F1 scores': array([0.67501011, 0.67112855, 0.67797704, 0.68316913, 0.67329304]), 'Mean Training F1 Score': 0.6761155736356493, 'Validation Accuracy scores': array([0.63714637, 0.63099631, 0.56703567, 0.60837438, 0.60714286]), 'Mean Validation Accuracy': 61.01391186325657, 'Validation Precision scores': array([0.67089297, 0.66607368, 0.62970445, 0.63741019, 0.65218723]), 'Mean Validation Precision': 0.651253702881182, 'Validation Recall scores': array([0.63714637, 0.63099631, 0.56703567, 0.60837438, 0.60714286]), 'Mean Validation Recall': 0.610

In [100]:
model_name = 'KNN'
knn_model = get_knn_model()
run_model(knn_model,x_train, y_train,x_test,y_test,model_name)
run_model_with_cv(model_name,knn_model,x_train, y_train)

Model: KNN  Accuracy score: 0.6401869158878505


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

KNN {'Training Accuracy scores': array([0.77846154, 0.77384615, 0.79076923, 0.80375269, 0.78037527]), 'Mean Training Accuracy': 78.54409767408845, 'Training Precision scores': array([0.8213368 , 0.81078295, 0.82456927, 0.83006074, 0.82300065]), 'Mean Training Precision': 0.8219500828072525, 'Training Recall scores': array([0.77846154, 0.77384615, 0.79076923, 0.80375269, 0.78037527]), 'Mean Training Recall': 0.7854409767408845, 'Training F1 scores': array([0.77563579, 0.76787953, 0.78732757, 0.80007051, 0.77973992]), 'Mean Training F1 Score': 0.7821306668159582, 'Validation Accuracy scores': array([0.68142681, 0.69126691, 0.64575646, 0.6637931 , 0.65640394]), 'Mean Validation Accuracy': 66.77294457673642, 'Validation Precision scores': array([0.73529071, 0.74102244, 0.70200333, 0.71419803, 0.70478773]), 'Mean Validation Precision': 0.7194604480250186, 'Validation Recall scores': array([0.68142681, 0.69126691, 0.64575646, 0.6637931 , 0.65640394]), 'Mean Validation Recall': 0.667729445767

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Training Accuracy scores': array([0.77846154, 0.77384615, 0.79076923, 0.80375269, 0.78037527]),
 'Mean Training Accuracy': 78.54409767408845,
 'Training Precision scores': array([0.8213368 , 0.81078295, 0.82456927, 0.83006074, 0.82300065]),
 'Mean Training Precision': 0.8219500828072525,
 'Training Recall scores': array([0.77846154, 0.77384615, 0.79076923, 0.80375269, 0.78037527]),
 'Mean Training Recall': 0.7854409767408845,
 'Training F1 scores': array([0.77563579, 0.76787953, 0.78732757, 0.80007051, 0.77973992]),
 'Mean Training F1 Score': 0.7821306668159582,
 'Validation Accuracy scores': array([0.68142681, 0.69126691, 0.64575646, 0.6637931 , 0.65640394]),
 'Mean Validation Accuracy': 66.77294457673642,
 'Validation Precision scores': array([0.73529071, 0.74102244, 0.70200333, 0.71419803, 0.70478773]),
 'Mean Validation Precision': 0.7194604480250186,
 'Validation Recall scores': array([0.68142681, 0.69126691, 0.64575646, 0.6637931 , 0.65640394]),
 'Mean Validation Recall': 0.667

In [101]:
model_name = 'Random Forest'
rf_model = get_random_forest_model()
run_model(rf_model,x_train, y_train,x_test,y_test,model_name)
run_model_with_cv(model_name,rf_model,x_train, y_train)

Model: Random Forest  Accuracy score: 0.7429906542056075


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

Random Forest {'Training Accuracy scores': array([0.99969231, 1.        , 0.99969231, 0.9996924 , 1.        ]), 'Mean Training Accuracy': 99.98154035444715, 'Training Precision scores': array([0.9996942 , 1.        , 0.99969433, 0.99969443, 1.        ]), 'Mean Training Precision': 0.9998165906731111, 'Training Recall scores': array([0.99969231, 1.        , 0.99969231, 0.9996924 , 1.        ]), 'Mean Training Recall': 0.9998154035444715, 'Training F1 scores': array([0.99969228, 1.        , 0.99969231, 0.99969234, 1.        ]), 'Mean Training F1 Score': 0.9998153864854362, 'Validation Accuracy scores': array([0.73431734, 0.7601476 , 0.72570726, 0.72413793, 0.73275862]), 'Mean Validation Accuracy': 73.5413750689231, 'Validation Precision scores': array([0.73245489, 0.75116997, 0.72358047, 0.73351149, 0.72553002]), 'Mean Validation Precision': 0.7332493691297236, 'Validation Recall scores': array([0.73431734, 0.7601476 , 0.72570726, 0.72413793, 0.73275862]), 'Mean Validation Recall': 0.735

{'Training Accuracy scores': array([0.99969231, 1.        , 0.99969231, 0.9996924 , 1.        ]),
 'Mean Training Accuracy': 99.98154035444715,
 'Training Precision scores': array([0.9996942 , 1.        , 0.99969433, 0.99969443, 1.        ]),
 'Mean Training Precision': 0.9998165906731111,
 'Training Recall scores': array([0.99969231, 1.        , 0.99969231, 0.9996924 , 1.        ]),
 'Mean Training Recall': 0.9998154035444715,
 'Training F1 scores': array([0.99969228, 1.        , 0.99969231, 0.99969234, 1.        ]),
 'Mean Training F1 Score': 0.9998153864854362,
 'Validation Accuracy scores': array([0.73431734, 0.7601476 , 0.72570726, 0.72413793, 0.73275862]),
 'Mean Validation Accuracy': 73.5413750689231,
 'Validation Precision scores': array([0.73245489, 0.75116997, 0.72358047, 0.73351149, 0.72553002]),
 'Mean Validation Precision': 0.7332493691297236,
 'Validation Recall scores': array([0.73431734, 0.7601476 , 0.72570726, 0.72413793, 0.73275862]),
 'Mean Validation Recall': 0.7354

In [ ]:
model_name = 'SVM Poly'
svm_poly = get_svm_model()
run_model(svm_poly,x_train, y_train,x_test,y_test,model_name)
run_model_with_cv(model_name,svm_poly,x_train, y_train)

Model: SVM Poly  Accuracy score: 0.7663551401869159


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
image_path = '/samples/sample1.png'
final_model = svm_poly
sound_file,predictions = recognize(final_model,image_path,show_sample=10)
print(" ".join(predictions))
Audio(sound_file, autoplay=True)
